In [ ]:
from stmpy import Driver, Machine
import ipywidgets as widgets
from IPython.display import display
import paho.mqtt.client as mqtt
import time

progressDict = {}
queue = []

mqttBroker = "mqtt.eclipseprojects.io"
subscriber = mqtt.Client("TA")
subscriber.connect(mqttBroker)

class TA:
    def __init__(self):
        print("Init - TA")
        self.button_help = widgets.Button(description = "Help next")

    def show_group_progress(self):
        print("Progress", progressDict)
        print("Queue", queue)

    def help_pressed(self, b):
        self.stm.send('help_finished')

    def help_group(self):
        print("Helping group {}".format(queue[0]))

#for TA
t0 = {"source": "initial", "target": "waiting"}
t1 = {
    "trigger": "help",
    "source": "waiting",
    "target": "assist"
}
t2 = {
    "trigger": "help_finished",
    "source": "assist",
    "target": "waiting"
}
t3 = {
    "trigger": "update",
    "source": "waiting",
    "target": "waiting"
}

#TA states
waiting = {'name': 'waiting',
      'entry': 'show_group_progress'
      }
assist = {'name': 'assist',
      'entry': 'help_group'
      }

ta = TA()
ta_machine = Machine(name='stm_ta', transitions=[t0, t1, t2, t3], obj=ta, states=[waiting, assist])
ta.stm = ta_machine

driver = Driver()
driver.add_machine(ta_machine)
driver.start()


def on_message(client, userdata, message):
    # Decodes the message to readable string
    msg = message.payload.decode("utf-8")
    if message.topic == "Help":
        # adds a group to queue
        queue.append(msg)
        ta.stm.send("help")
    elif message.topic == "Help_finished":
        # removes a group from queue
        queue.remove(msg)
        ta.stm.send("help_finished")
    elif message.topic == "Task" or message.topic == "Deliver":
        # split message to group and task number, for tracking progress
        msg = msg.split(" ")
        progressDict[msg[0]] = msg[1]
        ta.stm.send("update")

# A loop running and subscriber (TA) subscribes to different topics
subscriber.loop_start()
subscriber.subscribe("Help")
subscriber.subscribe("Help_finished")
subscriber.subscribe("Task")
subscriber.subscribe("Deliver")

# Triggered each time a message is published
subscriber.on_message = on_message

# The length of the loop
time.sleep(600)
subscriber.loop_stop()

driver.stop()